In [5]:
from src.schemas.thesaurus.mads import SchemaMads

from src.schemas.settings import Settings
from rdflib import URIRef, Namespace, Graph, Literal, BNode
from rdflib.namespace import RDF, RDFS
settings = Settings()

In [1]:
from src.schemas.catalog.work import BfWork
from datetime import datetime

In [2]:
req = {
    "adminMetadata": {
        "status": {
            "value": "http://id.loc.gov/vocabulary/mstatus/n",
            "label": "Novo"
        },
        "descriptionConventions": {
            "value": "http://id.loc.gov/vocabulary/descriptionConventions/aacr",
            "label": "AACr"
        }
    },
    "resource": [
        {
            "type": {
                "value": "Work",
                "label": "Obra"
            }
        },
        {
            "type": {
                "value": "Text",
                "label": "Texto"
            }
        }
    ],
    "language": [
        {
            "lang": {
                "value": "por",
                "label": "Português"
            }
        }
    ],
    "classification": {
        "cdd": "548.5",
        "cutter": "C456a"
    },
    "title": {
        "mainTitle": "Sol de feira",
        "subtitle": "Fábulas cruéis e outras estórias más"
    },
    "contribution": [
        {
            "authority": {
                "base": "bk",
                "value": "https://bibliokeia.com/authority/79",
                "label": "Cunha, Euclides da"
            },
            "role": {
                "value": "http://id.loc.gov/vocabulary/relators/aut",
                "label": "Autor"
            }
        }
    ],
    "subject": [
        {
            "type": {
                "value": "Topic",
                "label": "Termo Tópico"
            },
            "authority": {
                "base": "bk",
                "value": "https://bibliokeia.com/authority/80",
                "label": "Amazon River"
            },
            "lang": {
                "value": "por",
                "label": "Português"
            }
        }
    ]
}

In [3]:
request = BfWork(**req)
request.resource

[Resource(type=BfElement(value='Work', metadata=None, label='Obra')),
 Resource(type=BfElement(value='Text', metadata=None, label='Texto'))]

In [4]:
now = datetime.now()
request.adminMetadata.creationDate = now
request.adminMetadata.identifiedBy = 'TESTE'

In [9]:
request.resource


[Resource(type=BfElement(value='Work', metadata=None, label='Obra')),
 Resource(type=BfElement(value='Text', metadata=None, label='Texto'))]

In [6]:
from src.function.catalog.work.graphWork import MakeGraphWork


graph = MakeGraphWork(request)

In [7]:
print( graph)

@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://bibliokeia.com/works/TESTE> a <Text>,
        <Work> ;
    bf:adminMetadata [ a bf:AdminMetadata ;
            bf:assigner <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
            bf:creationDate "2024-06-03"^^xsd:date ;
            bf:descriptionConventions <http://id.loc.gov/vocabulary/descriptionConventions/aacr> ;
            bf:generationProcess [ a bf:GenerationProcess ;
                    rdfs:label "BiblioKeia" ;
                    bf:generationDate "2024-06-03T16:01:20"^^xsd:dateTime ] ;
            bf:identifiedBy [ a bf:Local ;
                    bf:assigner <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
                    rdf:value "TESTE" ] ;
            bf:status <http://id.loc.gov/vocabulary/mstatus/n> ] ;
    bf:class

In [7]:
MDASRDF = Namespace("http://www.loc.gov/mads/rdf/v1#")
BF = Namespace("http://id.loc.gov/ontologies/bibframe/")
IDENTIFIERS = Namespace("http://id.loc.gov/vocabulary/identifiers/")
identifier = f"{settings.base_url}/authorities/{request.adminMetadata.identifiedBy}"
resource = URIRef(identifier)

g = Graph(identifier=identifier)
g.bind('madsrdf', MDASRDF)
g.bind('identifiers', IDENTIFIERS)
g.bind('bf', BF)


In [13]:
from pysolr import Solr
solr = Solr(f'{settings.solr}/solr/authority/', timeout=10)

In [14]:
responseSolr = solr.add([doc], commit=True)
responseSolr

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":146\n  }\n}'

In [4]:
from src.function.thesaurus.solr.updateSolr import UpdateSolr


UpdateSolr(request)

In [8]:
graph = Graph()
graph.parse('http://id.loc.gov/authorities/names/no97027235.rdf')
graph.serialize('Graeber.ttl')

<Graph identifier=Nb33df10863d54bdf9a9ceeaf5d15d56d (<class 'rdflib.graph.Graph'>)>